# *Can't Stop* - Analysis

The board game *Can't Stop* allows users to roll 4 dice, and then choose
how to group them into two pairs.  The sum of each pair of dice are then
used as the "point" values which the user can advance one of his (up to
3) markers.

In each turn, a user may roll the dice as many times as they like - but they
must be able to advance one of their markers on each roll, otherwise they "crap out".

# Basic Questions

- What is the probability of rolling any particular point value (from 2 to 12) for
  any roll of 4 dice?
- What is the probability of "crapping out" for any particular set of 3 available point
  values?
  
  
Perhaps the most straightforward way to answer these questions is by exhastive simulation
of 4 dice rolls (which only require $6^4 = 1296$ rolls).

In [1]:
from itertools import product

In [2]:
dice = (1,2,3,4,5,6)
all_rolls = list(product(dice, dice, dice, dice))
len(all_rolls)

1296

In [3]:
def points(dice):
    """ Return set of point values that can be derived from a given (a pair of dice)
        in a roll of the dice.
    """
    result = set()
    num_dice = len(dice)
    for first in range(num_dice):
        for second in range(first+1, num_dice):
            result.add(dice[first] + dice[second])
            
    return result

In [4]:
all_points = [points(x) for x in all_rolls]

In [5]:
def prob_point(p):
    c = 0
    for points in all_points:
        if p in points:
            c += 1
    return c / len(all_points)

In [6]:
for p in range(2, 13):
    print(f"{p:2d}: {prob_point(p):.0%}")

 2: 13%
 3: 23%
 4: 36%
 5: 45%
 6: 56%
 7: 64%
 8: 56%
 9: 45%
10: 36%
11: 23%
12: 13%


# Balance

We can now answer how well balanced *Can't Stop* is.  To *capture* the 2 point
you need to book 3 twos.  Then 5 threes, 7 fours, etc:

| Point | # Needed |
| --- | --- |
| 2 | 3 |
| 3 | 5 |
| 4 | 7 |
| 5 | 9 |
| 6 | 11 |
| 7 | 13 |
| 8 | 11 |
| 9 | 9 |
| 10 | 7 |
| 11 | 5 |
| 12 | 3 |

If we divide the number of rolls needed by the probability of getting a specific point,
that will give us the expected number of rolls to capture each point.


In [7]:
for p in range(2, 13):
    needed = 13 - 2 * abs(7 - p)
    print(f"Point {p:2d} needs {needed:2d}: expected in {needed/prob_point(p):.1f} rolls")

Point  2 needs  3: expected in 22.7 rolls
Point  3 needs  5: expected in 21.5 rolls
Point  4 needs  7: expected in 19.7 rolls
Point  5 needs  9: expected in 20.1 rolls
Point  6 needs 11: expected in 19.6 rolls
Point  7 needs 13: expected in 20.2 rolls
Point  8 needs 11: expected in 19.6 rolls
Point  9 needs  9: expected in 20.1 rolls
Point 10 needs  7: expected in 19.7 rolls
Point 11 needs  5: expected in 21.5 rolls
Point 12 needs  3: expected in 22.7 rolls



# Point Sets

While realizing that you're 64% chance of rolling a 7, it's more
useful strategically to know how likely you are to "crap out" once
you've already established the 3 point's you're working toward.

In [8]:
point_sets = list(combinations(range(2, 13), 3))

NameError: name 'combinations' is not defined

In [ ]:
def prob_point_set(ps):
    c = 0
    for points in all_points:
        for pts in ps:
            if pts in points:
                c += 1
                break;
    return c / len(all_points)
        

In [ ]:
prob_point_set([6, 7, 8])

0.9197530864197531

In [ ]:
for ps in point_sets:
    print(f"{str(ps):>15s}: {prob_point_set(ps):.0%} ")

      (2, 3, 4): 52% 
      (2, 3, 5): 58% 
      (2, 3, 6): 68% 
      (2, 3, 7): 75% 
      (2, 3, 8): 76% 
      (2, 3, 9): 71% 
     (2, 3, 10): 63% 
     (2, 3, 11): 53% 
     (2, 3, 12): 44% 
      (2, 4, 5): 66% 
      (2, 4, 6): 76% 
      (2, 4, 7): 81% 
      (2, 4, 8): 82% 
      (2, 4, 9): 76% 
     (2, 4, 10): 74% 
     (2, 4, 11): 63% 
     (2, 4, 12): 55% 
      (2, 5, 6): 77% 
      (2, 5, 7): 81% 
      (2, 5, 8): 83% 
      (2, 5, 9): 76% 
     (2, 5, 10): 76% 
     (2, 5, 11): 71% 
     (2, 5, 12): 63% 
      (2, 6, 7): 86% 
      (2, 6, 8): 88% 
      (2, 6, 9): 83% 
     (2, 6, 10): 81% 
     (2, 6, 11): 76% 
     (2, 6, 12): 74% 
      (2, 7, 8): 89% 
      (2, 7, 9): 84% 
     (2, 7, 10): 83% 
     (2, 7, 11): 78% 
     (2, 7, 12): 78% 
      (2, 8, 9): 82% 
     (2, 8, 10): 82% 
     (2, 8, 11): 74% 
     (2, 8, 12): 74% 
     (2, 9, 10): 71% 
     (2, 9, 11): 64% 
     (2, 9, 12): 63% 
    (2, 10, 11): 58% 
    (2, 10, 12): 55% 
    (2, 11, 12): 44% 
      (3, 

# Strategy

If you have the easiest triple, of (6,7,8), you have a nearly 92% chance of making a point
(only an 8% chance of crapping out).  Given that once you have established your points, you've
already made 3 or 4 units of progress, you expect to lose $8\% \times n$ units, but you stand to gain
1 or 2 units by rolling again.  You maximize the expected gain when $92\% \times 1 > 8\% \times n$ (
or $n < 12$)!

On the other hand, if rolling on a (2, 3, 12), you have just 44% chance of making a point.  So you should
roll when $44\% > 64\% \times n$ (or $n < 0.6$).

In [ ]:
for s in range(44, 93, 6):
    p = s / 100
    print(f"{s}% - roll when n < {p/(1-p):1.0f}")

44% - roll when n < 1
50% - roll when n < 1
56% - roll when n < 1
62% - roll when n < 2
68% - roll when n < 2
74% - roll when n < 3
80% - roll when n < 4
86% - roll when n < 6
92% - roll when n < 12


Alternately, only when $\frac{p}{1-p} > 3$ should you ever continue rolling past the initial three points (i.e., $p > 0.75$).


In [ ]:
for ps in point_sets:
    p = prob_point_set(ps)
    n = p / (1 - p)
    if p > 0.75:
        print(f"{str(ps):>15s}: {p:.0%} - up to {n:.0f} at risk.")

      (2, 3, 7): 75% - up to 3 at risk.
      (2, 3, 8): 76% - up to 3 at risk.
      (2, 4, 6): 76% - up to 3 at risk.
      (2, 4, 7): 81% - up to 4 at risk.
      (2, 4, 8): 82% - up to 4 at risk.
      (2, 4, 9): 76% - up to 3 at risk.
      (2, 5, 6): 77% - up to 3 at risk.
      (2, 5, 7): 81% - up to 4 at risk.
      (2, 5, 8): 83% - up to 5 at risk.
      (2, 5, 9): 76% - up to 3 at risk.
     (2, 5, 10): 76% - up to 3 at risk.
      (2, 6, 7): 86% - up to 6 at risk.
      (2, 6, 8): 88% - up to 8 at risk.
      (2, 6, 9): 83% - up to 5 at risk.
     (2, 6, 10): 81% - up to 4 at risk.
     (2, 6, 11): 76% - up to 3 at risk.
      (2, 7, 8): 89% - up to 8 at risk.
      (2, 7, 9): 84% - up to 5 at risk.
     (2, 7, 10): 83% - up to 5 at risk.
     (2, 7, 11): 78% - up to 4 at risk.
     (2, 7, 12): 78% - up to 4 at risk.
      (2, 8, 9): 82% - up to 5 at risk.
     (2, 8, 10): 82% - up to 4 at risk.
      (3, 4, 7): 79% - up to 4 at risk.
      (3, 4, 8): 80% - up to 4 at risk.
